In [13]:
import pandas as pd
import json
from datetime import datetime

In [73]:
nov30_rens = pd.read_csv('carbon_intensity_pricing/raw CAISO/CAISO-renewables-20221130.csv', index_col=0, header=None).T
nov30_other = pd.read_csv('carbon_intensity_pricing/raw CAISO/CAISO-supply-20221130.csv', index_col=0, header=None).T

dec1_rens = pd.read_csv('carbon_intensity_pricing/raw CAISO/CAISO-renewables-20221201.csv', index_col=0, header=None).T
dec1_other = pd.read_csv('carbon_intensity_pricing/raw CAISO/CAISO-supply-20221201.csv', index_col=0, header=None).T

nov30_rens.rename(columns={nov30_rens.columns[0]: 'Time'}, inplace=True)
nov30_other = nov30_other.rename(columns={nov30_other.columns[0]: 'Time'}, inplace=False).dropna()

dec1_rens.rename(columns={dec1_rens.columns[0]: 'Time'}, inplace=True)
dec1_other = dec1_other.rename(columns={dec1_other.columns[0]: 'Time'}, inplace=False).dropna()


nov30_rens['Time'] = [datetime.combine(datetime(2022, 11, 30), time) for time in pd.to_datetime(nov30_rens['Time']).dt.time.tolist()]
nov30_other['Time'] = [datetime.combine(datetime(2022, 11, 30), time) for time in pd.to_datetime(nov30_other['Time']).dt.time.tolist()]

dec1_rens['Time'] = [datetime.combine(datetime(2022, 12, 1), time) for time in pd.to_datetime(dec1_rens['Time']).dt.time.tolist()]
dec1_other['Time'] = [datetime.combine(datetime(2022, 12, 1), time) for time in pd.to_datetime(dec1_other['Time']).dt.time.tolist()]


# combine the two dataframes on the time column
nov30 = pd.merge(nov30_rens, nov30_other, on='Time')
dec1 = pd.merge(dec1_rens, dec1_other, on='Time')

# concatenate the two dataframes
full = pd.concat([nov30, dec1], axis=0)



full.set_index('Time', inplace=True)

# convert all values to floats
full = full.astype(float)
# set all values in the dataframe less than 0 to 0
full[full < 0] = 0

# get rows between 9pm PST November 30th and 9pm PST December 1st
full = full.loc['2022-11-30 21:00:00':'2022-12-01 20:00:00']

# calculate the hourly average values
full = full.resample('H').mean()

CARBON_INTENSITIES_CAISO = json.load(open('CAISO_carbon_intensity.json', 'r'))

total = full["Coal"] + full["Natural gas"] + full["Wind"] + full["Nuclear"] + full["Solar"] + full["Small hydro"] + full["Large hydro"] + full["Biomass"] + full["Geothermal"] + full["Biogas"] + full["Imports"]

full["g CO2 e/kWh"] = (1/total)*(full["Coal"] * CARBON_INTENSITIES_CAISO["Coal"] + full["Natural gas"] * CARBON_INTENSITIES_CAISO["Natural gas"] \
    + full["Wind"] * CARBON_INTENSITIES_CAISO["Wind"] + full["Nuclear"] * CARBON_INTENSITIES_CAISO["Nuclear"] + full["Solar"] * CARBON_INTENSITIES_CAISO["Solar"] \
         + full["Small hydro"] * CARBON_INTENSITIES_CAISO["Small hydro"] + full["Large hydro"] * CARBON_INTENSITIES_CAISO["Large hydro"] + full["Biomass"] * CARBON_INTENSITIES_CAISO["Biomass"] \
            + full["Geothermal"] * CARBON_INTENSITIES_CAISO["Geothermal"] + full["Biogas"] * CARBON_INTENSITIES_CAISO["Biogas"] + full["Imports"] * CARBON_INTENSITIES_CAISO["Imports"]) 
print(full["g CO2 e/kWh"])





Time
2022-11-30 21:00:00    312.935539
2022-11-30 22:00:00    306.537738
2022-11-30 23:00:00    297.905380
2022-12-01 00:00:00    291.140733
2022-12-01 01:00:00    294.176349
2022-12-01 02:00:00    288.061553
2022-12-01 03:00:00    289.008076
2022-12-01 04:00:00    286.510766
2022-12-01 05:00:00    276.020398
2022-12-01 06:00:00    272.846030
2022-12-01 07:00:00    265.039352
2022-12-01 08:00:00    251.860794
2022-12-01 09:00:00    250.982116
2022-12-01 10:00:00    240.248128
2022-12-01 11:00:00    234.398249
2022-12-01 12:00:00    222.490436
2022-12-01 13:00:00    223.129334
2022-12-01 14:00:00    240.844175
2022-12-01 15:00:00    275.242945
2022-12-01 16:00:00    290.790090
2022-12-01 17:00:00    291.212044
2022-12-01 18:00:00    287.178583
2022-12-01 19:00:00    284.211986
2022-12-01 20:00:00    285.431130
Freq: H, Name: g CO2 e/kWh, dtype: float64
